# Module 8 Challenge

## Deliverable 2

extract and transform the Wikipedia data so you can merge it with the Kaggle metadata. While extracting the IMDb IDs using a regular expression string and dropping duplicates, use a try-except block to catch errors.

Outputs:
wiki_movies_df (cleaned wiki movies data)

In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time


In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy

    #combine alternate titles into one list
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
            
    change_column_name('Directed by', 'Director')
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie   

In [3]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)
#print('step 2')
def extract_transform_load(wiki_data,meta_data,ratings_data):
    
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
   
    kaggle_metadata = pd.read_csv(f'{meta_data}', low_memory=False)
    ratings = pd.read_csv(f'{ratings_data}')

    # Open and read the Wikipedia data JSON file.
    
    with open(f'{wiki_data}', mode='r') as file:
        wiki_movies_raw = json.load(file)
    #print(f'length wiki_movies_raw:  {len(wiki_movies_raw)}')
    
    # 3. Write a list comprehension to filter out TV shows.
    #print('step 3')
    wiki_movies = [movie for movie in wiki_movies_raw 
                   if (('Director' in movie or 'Directed by' in movie) 
                   and ('imdb_link' in movie) 
                   and ('No. of episodes' not in movie))]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    #print('step 4')
    cleaned_wiki_movies = [clean_movie(movie) for movie in wiki_movies]
    print(f'length of cleaned_wiki_movies: {len(cleaned_wiki_movies)}')
    
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    #print('step 5')
    wiki_movies_df = pd.DataFrame(cleaned_wiki_movies)
    print(f'length of wiki_movies:  len(wiki_movies_df)')
    
    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    #print('step 6')
    wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})') # this should not generate errors
    wiki_movies_df = wiki_movies_df.dropna(subset=['imdb_id']) # this should not generate errors
    #print(f'dropped rows with missing imdb_id. length of wiki_movies_df: {len(wiki_movies_df)}')
    try:
        wiki_movies_df.set_index('imdb_id',inplace=True,verify_integrity=True)
    except ValueError: 
        print('dropping duplicate imdb ids')
        dupes = wiki_movies_df.pivot_table(index=['imdb_id'],aggfunc=['size'])
        dupes = dupes[dupes['size']>1]
        print(dupes)
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        print(f'dropped duplicate imdb_df:  {len(wiki_movies_df)}')
    
    #  7. Write a list comprehension to keep the columns that have non-null values from the wiki_movies_df DataFrame.
    #print('step 7')
    #print('list comprehension keeping columns with any non-nulls')
    
    #list comprehension:
    # instead of a list of lists, create a named index list, column name is the index
    # <expression> for <item> in <list> if <conditional>
    # conditional: none
    # iterator: for var in wiki_movies_df.columns
    # expression: [var,wiki_movies_df[var].isnull().sum()]
    #print(f'columns with count of non-nulls')
    #print([[column,wiki_movies_df[column].notnull().sum()] for column in wiki_movies_df.columns])
    
    # now make a list comprehension to make a list of the columns that have non-nulls
    # [<expression> for <item> in <list> if <conditional>] 
    # conditional: if (wiki_movies_df[var].notnull().sum()>0
    # iterator: for var in wiki_movies_df.columns
    # expression: var
    # NOTE: directions say to keep columns with ANY non-null values, but that results in too many columns at the end.
    # Apparently this was supposed to be the same as in the module activities: 
    #  calculate the percentage of null values in the column
    #   if the percentage is < 90 add the column to our list of columns to keep 
    
    #wiki_columns_to_keep = [var for var in wiki_movies_df.columns if (wiki_movies_df[var].notnull().sum() > 0)]
    wiki_columns_to_keep = [var for var in wiki_movies_df.columns 
                            if (wiki_movies_df[var].isnull().sum() / len(wiki_movies_df))< 0.9]
    #print(f'columns with non-null values: count: {len(wiki_columns_to_keep)} columns: {wiki_columns_to_keep}')
    
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    #print(f'wiki_movies_df shape:  {wiki_movies_df.shape}')

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    #print('step8')  
    
    box_office = wiki_movies_df['Box office'].dropna()
    #print(f'box_office data type: {type(box_office)}, length: {len(box_office)}')
       
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    #print('step 9')
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    #print('step 10')
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
   
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    #print('step 11')
    form_two= r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

    # 12. Add the parse_dollars function.
    #print('step 12')
    # change strings into numerics using a parsing function
    # use re.match(pattern, string) to determine whether string matches the pattern)
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan
   
        # split the millions and billions patterns
        # form1: if input is of the form $###.# million
        # re.match returns the first match it finds in the string
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','',s)
            # convert to float and multiply by a million
            value=float(s) * 10**6
            # return value
            return value
   
        # form1: if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            # convert to float and multiply by a billion
            value = float(s) * 10**9
            # return value
            return value
    
        # form2: if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            # remove dollar sign and commas
            s = re.sub('\$|,','', s)
            # convert to float
            value = float(s)
            # return value
            return value
    
        # otherwise, return NaN
        else:
            return np.nan
     
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    #print('step 13')
    # fix number ranges by keeping just the upper end of the range
    # ends with zero to one period
    box_office = box_office.str.replace('\$.*[-—–](?![a-z])', '$', regex=True)
    
    # then extract only the parts of the strings that match the patterns 
    # combine into one capture group using pipe
    box_office.str.extract(f'({form_one}|{form_two})')
    
    # change strings into numerics using the parse dollars function
    
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', 
    flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    #drop the original Box Office column
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    #print(f'updated with clean box office numbers:  shape: {wiki_movies_df.shape}')
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    #print('step 14')
    budget=wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    #wiki_movies_df.drop('Budget', axis=1, inplace=True)
    
    #print(f'updated with clean budget numbers: shape: {wiki_movies_df.shape}')

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    #print('step 15')
    release_date=wiki_movies_df['Release date'].dropna().apply(lambda x:' '.join(x) if type(x) == list else x)
    form1 = r'[January|February|March|April|May|June|July|August|September|October|November|December][a-z]+\s\d{1,2},\s\d{4}'
    form2 = r'\d{4}[-—–]\d{2}[-—–]\d{2}'
    form3 = r'[January|February|March|April|May|June|July|August|September|October|November|December][a-z]+\s\d{4}'
    form4 = r'[1,2]\d{3}'
    wiki_movies_df['release_date'] =pd.to_datetime(release_date.str.extract(f'({form1}|{form2}|{form3}|{form4})',flags=re.IGNORECASE)[0],infer_datetime_format=True)
    #wiki_movies_df.drop('Release date', axis=1, inplace=True)
    
    print(f'updated with clean release dates: shape: {wiki_movies_df.shape}')
    
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    #print('step 16')
    running_time=wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x)==list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2]==0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    #print(f'updated with clean release dates: shape: {wiki_movies_df.shape}')
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [4]:
# 17. Create the path to your file directory and variables for the three files.
#print('step 17')
file_dir='c:/users/kathy/Desktop/GITRepositories/Movies-ETL/Resources'

# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
#print(wiki_file)
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
#print(kaggle_file)
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'
#print(ratings_file)


In [5]:
# 18. Set the three variables equal to the function created in D1.
#print('step 18')
(wiki_file, kaggle_file, ratings_file) = extract_transform_load(wiki_file, kaggle_file, ratings_file)

length of cleaned_wiki_movies: 7076
length of wiki_movies:  len(wiki_movies_df)
dropping duplicate imdb ids
           size
imdb_id        
tt0099180     2
tt0099816     2
tt0101414     2
tt0102432     2
tt0103002     2
tt0103923     2
tt0105226     2
tt0105616     2
tt0107978     2
tt0107983     2
tt0109266     2
tt0113646     2
tt0113855     2
tt0115819     2
tt0118577     2
tt0119256     2
tt0120241     2
tt0120338     2
tt0120915     2
tt0165831     2
tt0198781     2
tt0226168     2
tt0259446     2
tt0265307     2
tt0266543     2
tt0273982     2
tt0290212     2
tt0296192     2
tt0321780     3
tt0498381     2
tt0795351     2
tt0844286     2
tt1107319     2
tt1440379     2
tt1781840     2
tt1885300     2
tt1930294     2
tt1935896     2
tt2023765     2
tt2083379     2
tt2103264     2
tt2140577     2
dropped duplicate imdb_df:  7033
updated with clean release dates: shape: (7033, 23)


In [6]:
# 19. Set the wiki_movies_df equal to the wiki_file variable.
#print('step 19')
wiki_movies_df = wiki_file

In [9]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
#print('step 20')
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",US,English,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [10]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
#print('step 21')
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Country',
 'Language',
 'Budget',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id',
 'box_office',
 'budget',
 'release_date',
 'running_time']